In [1]:
import pandas as pd
TRAIN_FILEPATH = "../Translated/cleaned/train.csv"
TEST_FILEPATH = "../Translated/cleaned/test.csv"
SS_FILEPATH = "../data/SampleSubmission.csv"
VECTORS_FILEPATH = ""
train = pd.read_csv(TRAIN_FILEPATH)
test = pd.read_csv(TEST_FILEPATH)
ss = pd.read_csv(SS_FILEPATH)

C:\Users\amakr\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\amakr\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\amakr\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [81]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train.Text)
X_test_counts = count_vect.transform(test.Text)

X_train_counts.shape


(1436, 50468)

In [82]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_test_tf = tf_transformer.transform(X_test_counts)

X_train_tf.shape


(1436, 50468)

In [83]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

X_train_tfidf.shape

(1436, 50468)

In [84]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['label'] = le.fit_transform(train['Label'])

In [108]:
from sklearn.model_selection import KFold,StratifiedKFold,cross_val_score,train_test_split,StratifiedShuffleSplit

x_train, x_val = train_test_split(X_train_tfidf,test_size = 0.09, random_state = 42, stratify = train['label'])

In [110]:
x_train.shape

(1306, 50468)

In [86]:
from sklearn.naive_bayes import MultinomialNB, ComplementNB
clf = ComplementNB()
clf.fit(x_train,y_train)
clf.score(x_val,y_val)

0.5692307692307692

In [96]:
# x_val.shape
# y_val.unique().shape
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(loss='modified_huber', penalty='l2',
                          alpha=1e-5, random_state=1,
                          max_iter=1000, tol=None)
sgd_clf.fit(x_train,y_train)
sgd_clf.score(x_val,y_val)

0.676923076923077

In [106]:
from sklearn.svm import LinearSVC

svc_clf = LinearSVC(penalty='l2', dual=True,
                                       tol=1e-4)
svc_clf.fit(x_train,y_train)
svc_clf.score(x_val,y_val)

0.7076923076923077

In [75]:
# (le.inverse_transform(sgd_clf.predict(X_test_tfidf)))
test['Label'] = (le.inverse_transform(sgd_clf.predict(X_test_tfidf)))

In [76]:
sub = test[['ID','Label']]

In [77]:
sub.to_csv('sub_sgd3.csv',index = False)

In [107]:
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier